In [1]:
#coding=utf-8
import numpy as np
import pandas as pd

In [2]:
path = ('E:\\work\\竞争对手\\数据\\餐饮-计量\\jzqb_earlier6month.csv').decode('utf-8')
data_early = pd.read_csv(path, names=['shopid','list_user'])
data_early.head()

,shopid,list_user
0,1427469939,15026|1703212|5176157|5787401|6222925|6241420|...
1,1427495767,591220876|1101789123|1486286273|1544889055|172...
2,1427538754,1410509|2285814|2783100|14590476|15602920|2151...
3,1427550629,2129564|35210648|253893716|733200958|745665947...
4,1427582876,1096213|1410509|1478861|2285814|4469109|679745...


In [3]:
path = ('E:\\work\\竞争对手\\数据\\餐饮-计量\\jzqb_later6month.csv').decode('utf-8')
data_later = pd.read_csv(path)
data_later.head()

,店铺id,time,销量
0,1427469939,2015-11,484
1,1427469939,2015-12,294
2,1427469939,2016-01,218
3,1427469939,2016-02,112
4,1427469939,2016-03,162


In [4]:
# 生成list_shop
list_shop = list(set(data_early['shopid'].values))
# 生成dict{shopid: user_sacle}
dict_shop_scale = {}
for shop in list_shop:
    scale = len(set(data_early[data_early['shopid']==shop]['list_user'].values[0].split('|')))
    dict_shop_scale[shop] = scale

In [5]:
# 生成了两两竞争强度（用户重合度），并保存
# path = ('E:\\work\\竞争对手\\数据\\餐饮-计量\\coincide.csv').decode('utf-8')
# f = open(path, 'w')
# f.write('shop1,shop2,coincide\r\n')
# for shop1 in list_shop:
#     for shop2 in list_shop:
#         if shop1 == shop2: continue
#         user1 = set(data_early[data_early['shopid']==shop1]['list_user'].values[0].split('|'))
#         user2 = set(data_early[data_early['shopid']==shop2]['list_user'].values[0].split('|'))
#         f.write(str(shop1) + ',' + str(shop2) + ',' + str(len(user1 & user2)) + '\r\n')
# f.close()
# 读取两两竞争强度
path = ('E:\\work\\竞争对手\\数据\\餐饮-计量\\coincide.csv').decode('utf-8')
data_coincide = pd.read_csv(path)

In [6]:
# 每月市场大小
list_month = ['2015-11','2015-12','2016-01','2016-02','2016-03','2016-04','2016-05','2016-06','2016-07','2016-08','2016-09','2016-10']
temp = data_later.groupby('time').sum()
dict_month_scale = temp.to_dict()
print dict_month_scale['销量']

{'2016-07': 86481, '2016-06': 78189, '2016-05': 76441, '2016-04': 65260, '2016-03': 61718, '2016-02': 22925, '2016-01': 37808, '2015-11': 25513, '2016-10': 53571, '2015-12': 32135, '2016-09': 59773, '2016-08': 68247}


In [7]:
# 将业绩表现写入data_later
## 1.市场份额比例（每月销量占市场总销量的比例）
def getPerformance_1(x):
    return float(x[2]) / dict_month_scale['销量'][x[1]]

data_later['performance_1'] = data_later[['店铺id','time','销量']].apply(getPerformance_1,axis=1) 

In [8]:
data_later[:13]

,店铺id,time,销量,performance_1
0,1427469939,2015-11,484,0.018971
1,1427469939,2015-12,294,0.009149
2,1427469939,2016-01,218,0.005766
3,1427469939,2016-02,112,0.004885
4,1427469939,2016-03,162,0.002625
5,1427469939,2016-04,130,0.001992
6,1427469939,2016-05,103,0.001347
7,1427469939,2016-06,203,0.002596
8,1427469939,2016-07,344,0.003978
9,1427469939,2016-08,230,0.003370


In [19]:
# dict_shopid_shopname
import MySQLdb
# 连接数据库
conn=MySQLdb.connect(host='192.168.1.119',user='mysqlsmda',passwd='123456',db='smda',port=3306,charset='utf8')
sql = "SELECT shop_id, shop_name FROM smda.tb_baiduwaimaishop;"
data_shopidname = pd.read_sql(sql, conn)
keys = data_shopidname['shop_id'].values
values = data_shopidname['shop_name'].values
dict_shopid_shopname= dict(zip(keys, values))

## 1.面板数据实证

In [9]:
# 定义绩效
dict_performance = {}
def getPerformance(shopid, time):
    return performanceShare(shopid, time) #市场份额比例（每月销量占市场总销量的比例）
#     return performanceShareIncreaseRate(shopid, time) ## 市场份额增长率((本月份额-上月份额)/上月份额)


## 市场份额比例（每月销量占市场总销量的比例）
def performanceShare(shopid, time):
    sale = data_later[(data_later['店铺id']==shopid) & (data_later['time']==time)]['销量'].values[0]
    return float(sale) / dict_month_scale['销量'][time]
## 市场份额增长率((本月份额-上月份额)/上月份额)
def performanceShareIncreaseRate(shopid, time):
    last_time = list_month[list_month.index(time)-1]
    sale_now = data_later[(data_later['店铺id']==shopid) & (data_later['time']==time)]['销量'].values[0]
    sale_now = float(sale_now) / dict_month_scale['销量'][time]
    sale_last = data_later[(data_later['店铺id']==shopid) & (data_later['time']==last_time)]['销量'].values[0]
    sale_last = float(sale_last) / dict_month_scale['销量'][last_time]
    return (sale_now-sale_last) / sale_last

In [ ]:
# 每家企业的top3竞争对手
dict_shop_competitor = {}
## 按照重合度(等同于 重合度/A企业用户规模)
for shop in list_shop:
    competitors = list(data_coincide[data_coincide['shop1']==shop].sort_values(by='coincide',ascending=False)['shop2'].values[:3])
    dict_shop_competitor[shop] = competitors

In [ ]:
# 筛选list_shop(面板数据)，考虑自身与竞争对手销量
list_shop_fliter = []
sale_limit = 50 ###每月销量阈值
def noSale(shopid):
    ### 后六个月必须每个月都有销量，且每月销量大于n
    sales = data_later[(data_later['店铺id']==shopid) & (data_later['time'].isin(list_month[-6:]))]['销量'].values
    if len(sales) < 6 or min(sales) < sale_limit:
        return True
    return False

for shop in list_shop:
    if noSale(shop): continue
    list_competitor = dict_shop_competitor[shop]
    if noSale(list_competitor[0]): continue
    if noSale(list_competitor[1]): continue
    if noSale(list_competitor[2]): continue
    list_shop_fliter.append(shop)

print '筛选后剩余' + str(len(list_shop_fliter)) + '家店铺'

In [ ]:
# 输出 月份，shopid，竞争对手1绩效，竞争对手2绩效，竞争对手3绩效
path = ('E:\\work\\竞争对手\\数据\\餐饮-计量\\result.csv').decode('utf-8')
f = open(path, 'w')
f.write('time,shopid,performance,last,competitor1,competitor2,competitor3\r\n')
for shopid in list_shop_fliter:
    months = list_month[-6:]
    for index,time in enumerate(months):
        list_competitor = dict_shop_competitor[shopid]
        print str(time) + ',' + str(shopid) + ',' + str(getPerformance(shopid, time))
        f.write(time + ',' + str(shopid) + ',' + str(getPerformance(shopid, time)) + ','\
                + str(getPerformance(shopid, months[index-1])) + ',' + str(getPerformance(list_competitor[0], time)) + ','\
                + str(getPerformance(list_competitor[1], time)) + ',' + str(getPerformance(list_competitor[2], time)) + '\r\n')
f.close()

In [ ]:
temp = pd.read_csv(path)
temp.head()

## 2.相关关系实证

In [10]:
# 筛选list_shop(相关关系实证)，只考虑自身销量
list_shop_fliter = []
sale_limit = 50 ###每月销量阈值
def noSale(shopid):
    ### 后六个月必须每个月都有销量，且每月销量大于n
    sales = data_later[(data_later['店铺id']==shopid) & (data_later['time'].isin(list_month[-6:]))]['销量'].values
    if len(sales) < 6 or min(sales) < sale_limit:
        return True
    return False

for shop in list_shop:
    if noSale(shop): continue
    list_shop_fliter.append(shop)
    
print '筛选后剩余' + str(len(list_shop_fliter)) + '家店铺'

筛选后剩余140家店铺


In [11]:
data_filter_coincide = data_coincide[data_coincide['shop1'].isin(list_shop_fliter) & data_coincide['shop2'].isin(list_shop_fliter)].copy()

In [12]:
# 计算相对竞争强度（有向）
def getRelativeCoincideDirect(x):
    return float(x[1]) / dict_shop_scale[x[0]]
data_filter_coincide['coincide_relative_direct'] = data_filter_coincide[['shop2','coincide']].apply(getRelativeCoincideDirect, axis=1)

In [13]:
# 计算相对竞争强度（无向）
def getRelativeCoincideUnDirect(x):
    return float(x[2]) / (dict_shop_scale[x[0]] + dict_shop_scale[x[1]])
data_filter_coincide['coincide_relative_undirect'] = data_filter_coincide[['shop1','shop2','coincide']].apply(getRelativeCoincideUnDirect, axis=1)

In [14]:
# 计算绩效相关性（shop1与shop2）
from scipy.stats import pearsonr
def getCorrelation(x):
    sales1 = data_later[(data_later['店铺id']==x[0]) & (data_later['time'].isin(list_month[-6:]))]['performance_1'].values
    sales2 = data_later[(data_later['店铺id']==x[1]) & (data_later['time'].isin(list_month[-6:]))]['performance_1'].values
    return pearsonr(sales1, sales2)[0]
def getCorrelation_p(x):
    sales1 = data_later[(data_later['店铺id']==x[0]) & (data_later['time'].isin(list_month[-6:]))]['performance_1'].values
    sales2 = data_later[(data_later['店铺id']==x[1]) & (data_later['time'].isin(list_month[-6:]))]['performance_1'].values
    return pearsonr(sales1, sales2)[1]

data_filter_coincide['correlation'] = data_filter_coincide[['shop1','shop2']].apply(getCorrelation, axis=1)
data_filter_coincide['p_value'] = data_filter_coincide[['shop1','shop2']].apply(getCorrelation_p, axis=1)

In [55]:
# 写入shop1和shop2的店名
def getShopName(x):
    if dict_shopid_shopname.has_key(str(x)):
        return dict_shopid_shopname[str(x)]
    return '未知'
data_filter_coincide['shop1_name'] = data_filter_coincide['shop1'].apply(getShopName)
data_filter_coincide['shop2_name'] = data_filter_coincide['shop2'].apply(getShopName)

In [58]:
data_filter_coincide.sort_values(by='coincide_relative_undirect', ascending=False)

,shop1,shop2,coincide,coincide_relative_direct,coincide_relative_undirect,correlation,p_value,shop1_name,shop2_name
26414,1450617003,1480028335,296,0.325991,0.134545,0.475768,0.340194,未知,一家人（东川路）
26975,1480028335,1450617003,296,0.229102,0.134545,0.475768,0.340194,一家人（东川路）,未知
27410,1480028335,1508783899,216,0.293878,0.131467,0.047023,0.929517,一家人（东川路）,德食餐厅（东川路）
271011,1508783899,1480028335,216,0.237885,0.131467,0.047023,0.929517,德食餐厅（东川路）,一家人（东川路）
26537,1450617003,1434223343,378,0.214043,0.123610,0.394752,0.438628,未知,饭盒子（东川路店）
95978,1434223343,1450617003,378,0.292570,0.123610,0.394752,0.438628,饭盒子（东川路店）,未知
26849,1450617003,1508783899,245,0.333333,0.120868,0.603843,0.204324,未知,德食餐厅（东川路）
271010,1508783899,1450617003,245,0.189628,0.120868,0.603843,0.204324,德食餐厅（东川路）,未知
27452,1480028335,1427582876,249,0.207846,0.118234,-0.043768,0.934390,一家人（东川路）,未知
294573,1427582876,1480028335,249,0.274229,0.118234,-0.043768,0.934390,未知,一家人（东川路）


In [16]:
def getCorrelation_test(shop1, shop2):
    sales1 = data_later[(data_later['店铺id']==shop1) & (data_later['time'].isin(list_month[-6:]))]['performance_1'].values
    sales2 = data_later[(data_later['店铺id']==shop2) & (data_later['time'].isin(list_month[-6:]))]['performance_1'].values
#     print dict_shopid_shopname[shop1]
    print sales1
#     print dict_shopid_shopname[shop2]
    print sales2
    print pearsonr(sales1, sales2)

getCorrelation_test(1443537695, 1450617003)

[ 0.00325741  0.00277533  0.0014454   0.00159714  0.00120456  0.00145601]
[ 0.00391151  0.00470654  0.00289081  0.00285727  0.00334599  0.00209068]
(0.74719280107095554, 0.087788578647863783)


In [22]:
data_filter_coincide.head()

,shop1,shop2,coincide,coincide_relative_direct,coincide_relative_undirect,correlation,p_value,shop1_name
5,1459224577,1530517521,0,0.000000,0.000000,-0.309309,0.550833,未知
16,1459224577,1581522255,0,0.000000,0.000000,-0.001053,0.998421,未知
18,1459224577,1583716412,0,0.000000,0.000000,0.899729,0.014577,未知
19,1459224577,1551011901,1,0.001992,0.000549,0.812560,0.049408,未知
23,1459224577,1542250576,0,0.000000,0.000000,0.351760,0.494122,未知


In [ ]:
# 绘图
import matplotlib.pyplot as plt
temp = data_filter_coincide[(data_filter_coincide['coincide_relative_undirect']>=0.05) & (data_filter_coincide['p_value']<=1.00)]
plt.scatter(temp['coincide_relative_direct'],temp['correlation'])
plt.show()

## 3.网络生成

In [ ]:
# 点、线的阈值设置
threshold_node = 50
threshold_line = 10
threshold_line_per = 0.15

In [ ]:
# 生成dict_shopname_weight
dict_shopname_weight = {}
count = 0
for key in dict_shop_scale.keys():
    if dict_shopid_shopname.has_key(str(key)):
        if dict_shop_scale[key] >= threshold_node:
            shopname = dict_shopid_shopname[str(key)]
            dict_shopname_weight[shopname] = dict_shop_scale[key]
    else:
        count += 1
print '有' + str(count) + '个店家没有shopname'
print '剩余' + str(len(dict_shopname_weight.keys())) + '个店家'

In [ ]:
# 生成dict_shopnameline_weight
dict_shopnameline_weight = {}
dict_coincide = data_coincide[data_coincide['coincide']>threshold_line].to_dict('index') #线的绝对阈值
for i in dict_coincide.keys():
    if dict_shopid_shopname.has_key(str(dict_coincide[i]['shop1'])) and dict_shopid_shopname.has_key(str(dict_coincide[i]['shop2'])):
        line = dict_shopid_shopname[str(dict_coincide[i]['shop1'])] + ',' + dict_shopid_shopname[str(dict_coincide[i]['shop2'])]
        weight = float(dict_coincide[i]['coincide']) / dict_shop_scale[dict_coincide[i]['shop2']]
        if weight < threshold_line_per: continue #线的相对阈值
        dict_shopnameline_weight[line] = weight

In [ ]:
# 网络生成
import iNeedNet
path_ucinet = 'E:\work\竞争对手\数据\餐饮-网络\competitor_food.net'
path_gephi_node = 'E:\work\竞争对手\数据\餐饮-网络\competitor_food_node.csv'
path_gephi_line = 'E:\work\竞争对手\数据\餐饮-网络\competitor_food_line.csv'
iNeedNet.iNeedNet(dict_shopname_weight, dict_shopnameline_weight, True, False, path_ucinet, path_gephi_node, path_gephi_line)